In [1]:
import glob
import numpy as np
import pandas as pd

In this notebook I prepare data for a model where i use different lags to predict future prices in a specific index.

In [2]:
#**IMPORTING DATA**
files=glob.glob('../../data/price_index_data/mod/*')

#getting the prices history from each index
data=[]
for file in files:
    data.append(pd.read_csv(file))

#converting the 'date' columns to datetime type
for i in data:
    i['date']=pd.to_datetime(i['date'])

#combining the data together
df=data[0]
for i in data[1:]:
    df=df.merge(i,on='date')

#removing the date column and setting it as the index
df.index=df['date']
df.drop(columns=['date'],inplace=True)

#removing unecessary columns
cols=[i for i in list(df.columns) if 'Unnamed' not in i]
df=df[cols]
df=df.dropna()

In [3]:
targets=['bit_close','eth_close','ada_close','bnb_close']

There was a lot of auto correlation so different lags will help the linear model make its predictions.

In [4]:
features=[]
for target in targets:
    y=pd.DataFrame(df[target])
    for lags in range(1,8):
        y[f'lag_{lags}']=y[target].shift(lags)
    features.append(y.dropna())

In [5]:
for i,j in enumerate(features):
    j.to_csv(f'../../data/train/{targets[i]}_solo_linear_model.csv')